# Casos prácticos

Este notebook, by [Felipe Alonso Atienza](www.linkedin.com/in/felipe-alonso-atienza)

En este notebook vamos a abordar dos casos prácticos interesantes.

1. Predicción (*forecasting*) de la demanda de bicicletas
2. Clasificación multiclase de imágenes

## Librerías y funciones

Lo primero es cargar las librerías y funciones necesarias.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def plot_confusion_matrix(confmat):
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.5)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')

    plt.xlabel('predicted label')
    plt.ylabel('true label')

    plt.tight_layout()
    plt.show()

# 1. Predicción de la demanda de bicicletas

El problema está descrito [aquí](https://christophm.github.io/interpretable-ml-book/bike-data.html), y los datos pueden descargarse en la [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset), aunque los hemos modificado un poco para hacer el problema más parecido al [*challenge*](https://www.kaggle.com/c/bike-sharing-demand/data) original.

In [ ]:
data = pd.read_csv('./data/bikes.csv',sep=';', decimal='.')
data.head()

<div class = "alert alert-success">
**EJERCICIO**: Interprete los datos y realiza tus primeras hipótesis sobre qué variables son de interés
</div>

<div class = "alert alert-success">
**EJERCICIO**: Elimine las columnas *instant*, *casual* y *registered*
</div>

In [ ]:
# your code here


## 1.1 *Feature Engineering*

Vamos a trabajar con las fechas para crear algunas variables auxiliares

In [ ]:
from datetime import datetime

data['dteday'] = data['dteday'].apply(lambda x: datetime.strptime(x,'%d-%m-%Y'))
data.head()

In [ ]:
data['year'] = data['dteday'].apply(lambda x: x.year - 2011)
data.head()

In [ ]:
data['month'] = data['dteday'].apply(lambda x: x.month)
data.head()

In [ ]:
data['weekday'] = data['dteday'].apply(lambda x: x.isoweekday())
data.head()

Llegados a este punto, podemos eliminar la variable *dteday*

In [ ]:
data = data.drop(['dteday'],axis=1)
data.head()

## 1.2 Análisis de correlación

<div class = "alert alert-success">
**EJERCICIO**: Represente la variable *temp* vs *atemp*.
</div>

In [ ]:
# your code here


A la vista está que son variables altamente correlacionadas. Podemos eliminar *temp*, ya que nos afecta más la sensación térmica que la temperatura real.

In [ ]:
data = data.drop(['temp'],axis=1)
data.head()

<div class = "alert alert-success">
**EJERCICIO**(@home): Realice un análisis exploratorio exhaustivo.
</div>

# 1.3 Codificación de variables categóricas

Tenemos varias variables categóricas: *season*, *weathersit*, *month* y *weekday*. Cuando trabajamos con series temporales, es común crear variables *dummies* asociadas a cada una de las situaciones de las variables categóricas. Para ello, tenemos dos opciones:

- [pd.get_dummies()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html): se aplica directamente sobre el dataframe puede [utilizarse](https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40) con strings directamente. La pega es que genera un nuevo dataframe que hay que agregar al original.
- [OneHotEncoding()](http://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features): sobre la matriz de datos (X), genera variables dummies sobre las columnas que se indique. Es muy sencillo de implementar, pero requiere que los datos hayan sido codificados previamente como enteros (labelEncoder()).

La codificación "dummy" de variables categóricas en problemas de clasificación/regresión es opcional, como vimos anteriormente, pero en series temporales resulta adecuado para explicar el efecto de una situación temporal en la variable target.

Vamos a utilizar los dos métodos para codificar las variables categóricas.  

<div class = "alert alert-success">
**EJERCICIO**(@home): Prueba no realizar esta codificación "dummy" y entrena un modelo de machine learning para predecir la demanda de bicicletas.
</div>

Comenzamos por la variable *season*. Veamos qué hace *get_dummies()*.

In [ ]:
pd.get_dummies(data['season'], prefix = 'season')

In [ ]:
dummy = pd.get_dummies(data['season'], prefix = 'season')

data = pd.concat([data,dummy],axis=1).drop(['season'],axis=1)
data.head()

In [ ]:
# preparamos los datos
features = data.columns.drop(['cnt'])
X = data[features].as_matrix()
y = data['cnt'].as_matrix()

print('Filas, columnas', X.shape)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(categorical_features = [2,8,9], sparse=False, n_values=[3,12,7]) #weathersit, month, weekday
X = enc.fit_transform(X)
print('Filas, columnas', X.shape)

## 1.4 División train/test 

In [ ]:
# Paso 1:
offset = 182 # 0.25 of 731

X_train = X[:-offset, :]
y_train = y[:-offset]
X_test  = X[-offset:, :]
y_test  = y[-offset:]

plt.plot(range(0,len(y_train)),y_train, label='train')
plt.plot(range(len(y_train),len(y)),y_test,label='test')
plt.legend()
plt.show()

## 1.5 Búsqueda de parámetros libres

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import Lasso

tscv = TimeSeriesSplit(n_splits=3)

alpha_vector = np.logspace(-4,4,20)
param_grid = {'alpha': alpha_vector}

grid = GridSearchCV(Lasso(), param_grid=param_grid, cv = tscv.split(X_train)).fit(X_train, y_train)

In [ ]:
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))

scores = grid.cv_results_['mean_test_score']
std_scores = grid.cv_results_['std_test_score']
plt.errorbar(np.log10(alpha_vector),scores,yerr=std_scores, fmt='o-',ecolor='g')
plt.xlabel('log(alpha)',fontsize=16)
plt.ylabel('CV MSE')
plt.grid()
plt.show()

## 1.6 Métricas en test

In [ ]:
from sklearn.metrics import mean_squared_error

alpha_optimo = grid.best_params_['alpha']
lasso = Lasso(alpha = alpha_optimo).fit(X_train,y_train)

ytrainLasso = lasso.predict(X_train)
ytestLasso  = lasso.predict(X_test)

mseTrainModelLasso = mean_squared_error(y_train,ytrainLasso)
mseTestModelLasso = mean_squared_error(y_test,ytestLasso)

print('MSE Modelo Lasso (train): %0.3g' % mseTrainModelLasso)
print('MSE Modelo Lasso (test) : %0.3g' % mseTestModelLasso)

w = lasso.coef_
for f,wi in zip(features,w):
    print(f,wi)

<div class = "alert alert-success">
**EJERCICIO**: Representa la predicción obtenida junto con la serie real (train+test)
</div>

In [ ]:
# your code here


# 2. Clasificación multiclase de imágenes

En este caso vamos a utilizar la famosa base de datos de [MNIST](http://yann.lecun.com/exdb/mnist/). Esta base de datos contiene

* Training set: train-images-idx3-ubyte.gz (9.9 MB, 47 MB unzipped, 60,000 samples)
* Training set labels: train-labels-idx1-ubyte.gz (29 KB, 60 KB unzipped, 60,000 labels)
* Test set images: t10k-images-idx3-ubyte.gz (1.6 MB, 7.8 MB, 10,000 samples)
* Test set labels: t10k-labels-idx1-ubyte.gz (5 KB, 10 KB unzipped, 10,000 labels)

Estas imágenes se pueden descargar a partir del siguiente código (previamente hay que descargarse los archivos).

In [ ]:
import os
import struct
import numpy as np
import gzip
 
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, 
                               '%s-labels-idx1-ubyte.gz' % kind)
    images_path = os.path.join(path, 
                               '%s-images-idx3-ubyte.gz' % kind)
        
    with gzip.open(labels_path, 'rb') as lbpath:
        lbpath.read(8)
        buffer = lbpath.read()
        labels = np.frombuffer(buffer, dtype=np.uint8)

    with gzip.open(images_path, 'rb') as imgpath:
        imgpath.read(16)
        buffer = imgpath.read()
        images = np.frombuffer(buffer, 
                               dtype=np.uint8).reshape(
            len(labels), 784).astype(np.float64)
 
    return images, labels

In [ ]:
X_train, y_train = load_mnist('./data/mnist/', kind='train')
print('Rows: %d, columns: %d' % (X_train.shape[0], X_train.shape[1]))

In [ ]:
X_test, y_test = load_mnist('mnist/', kind='t10k')
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

No obstante, sklearn tiene la base de datos incluida en sus datasets.

In [ ]:
from sklearn.datasets import fetch_mldata

mnist = fetch_mldata("MNIST original")
# rescale the data, use the traditional train/test split
X, y = mnist.data / 255., mnist.target
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

¡Fíjate que el conjunto de entrenamiento son los pixels de la imagen tal cual!

In [ ]:
print ("X train shape: ", X_train.shape)
print ("y train shape: ", y_train.shape)
print ("X test shape: ",  X_test.shape)
print ("y test shape: ", y_test.shape)

Representamos algunos ejemplos:

In [ ]:
randomIds = np.random.randint(0,60000,10)

fig, ax = plt.subplots(nrows=2, ncols=5, sharex=True, sharey=True,)
ax = ax.flatten()
for i,n in enumerate(randomIds):
    img = X_train[n,:].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

<div class = "alert alert-success">
**EJERCICIO**: Entrena un modelo de regresión logística con C = 10 y calcula sus prestaciones en el conjunto de test. A lo mejor te resulta de utilidad revisar la [documentación](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
</div>

In [ ]:
from sklearn.linear_model import LogisticRegression

# your code here


<div class = "alert alert-success">
**EJERCICIO**: Calcula y representa la matriz de confusión, ¿qué conclusiones puedes sacar?
</div>

In [ ]:
from sklearn.metrics import confusion_matrix

# your code here
